# Composite nodes

One of the convenient properties of Forney-style factor graphs (as opposed to regular factor graphs) is that they naturally allow for composability: one can draw a box around part of an FFG and treat this box as a new type of factor node. For example, one can combine a gain (multiplication) node with the addition node in a so-called *composite node*, as depicted in the following graph (Fig. 4.2 from Korl's [A factor graph approach to signal modelling, system identification and filtering](https://www.research-collection.ethz.ch/handle/20.500.11850/82737)):

<img src="./figures/information_filter.png" width="350"/>

Composite nodes are useful for two reasons:

1. Building large graphs becomes more convenient by 'packaging' repetitive parts of the graph as composite nodes.
2. One can define 'shortcut rules' for message updates, which might be more efficient and/or numerically stable than performing vanilla message passing on the internals of the composite node. For example, in the schedule shown above, message (4) is calculated directly from messages (2) and (3). The shortcut rule might exploit the matrix inversion lemma, or involve some optimization algorithm.

To demonstrate the use of composite nodes, in this demo we will build a gain-addition combination that constrains

\begin{align*}
    x_1 = x_0 + b\times u_1\,,
\end{align*}

where `x_0` and `u_1` have Gaussian priors, and b is a constant. We are interested in computing a belief over `x_1`.

We first construct a "flat" graph that represents the gain and addition contraints as two distinct factors, and generate a schedule for inferring a belief for `x_1`. Then, we compare the resulting schedule with a schedule generated on an FFG with a _composite_ gain-addition node. Finally, we show how to register a custom update rule with ForneyLab.

## Schedule generation without composite node

In [1]:
using ForneyLab

# Define factor graph for x1 = x0 + b*u1, where x0 and u1 have Gaussian priors, and b is a constant.
# This is a part of the information filter graph from the introduction.
g = FactorGraph()

b = [1.0; 0.5]

@RV x_0 ~ GaussianMeanVariance(ones(2), eye(2))
@RV u_1 ~ GaussianMeanVariance(1.0, 1.0)
@RV x_1 = x_0 + b*u_1;

In [2]:
flat_schedule = sumProductSchedule(x_1)

draw(g, schedule=flat_schedule) # Inspect the resulting schedule

println(flat_schedule)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 16810182744494651225 -->
 
 16810182744494651225 
 
 clamp_1 
 
<!-- 882645485952632197 -->
 
 882645485952632197 
 
 clamp_2 
 
<!-- 6995855761247753794 -->
 
 6995855761247753794 
 
 clamp_3 
 
<!-- 16391658827319862511 -->
 
 16391658827319862511 
 
 clamp_4 
 
<!-- 1291578429769343262 -->
 
 1291578429769343262 
 
 + 
 addition_1 
 
<!-- 2268190176670543397 -->
 
 2268190176670543397 
 
 × 
 multiplication_1 
 
<!-- 1291578429769343262--2268190176670543397 -->
 
 1291578429769343262--2268190176670543397 
 
 variable_1 
 1 out 
 (3) 
 3 in2 
 
<!-- 13405660790449334050 -->
 
 13405660790449334050 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 1291578429769343262--13405660790449334050 -->
 
 1291578429769343262--13405660790449334050 
 
 x_0 
 1 out 
 (1) 
 2 in1 
 
<!-- 1726303011102148794 -->
 
 1726303011102148794 
 
 clamp_5 
 
<!-- 2268190176670543397--1726303011102148794 -->
 
 2268190176670543397--1726303011102148794 
 
 clamp_5 
 1 out 
 3 a 
 
<!-- 13787578128285340661 -->
 
 13787578128285340661 
 
 𝒩 
 gaussianmeanvariance_2 
 
<!-- 2268190176670543397--13787578128285340661 -->
 
 2268190176670543397--13787578128285340661 
 
 u_1 
 1 out 
 (2) 
 2 in1 
 
<!-- 13787578128285340661--6995855761247753794 -->
 
 13787578128285340661--6995855761247753794 
 
 clamp_3 
 1 out 
 2 m 
 
<!-- 13787578128285340661--16391658827319862511 -->
 
 13787578128285340661--16391658827319862511 
 
 clamp_4 
 1 out 
 3 v 
 
<!-- 13405660790449334050--16810182744494651225 -->
 
 13405660790449334050--16810182744494651225 
 
 clamp_1 
 1 out 
 2 m 
 
<!-- 13405660790449334050--882645485952632197 -->
 
 13405660790449334050--882645485952632197 
 
 clamp_2 
 1 out 
 3 v 
 
<!-- 182028522382400086022 -->
 
 182028522382400086022 
 
<!-- 182028522382400086022--1291578429769343262 -->
 
 182028522382400086022--1291578429769343262 
 
 x_1 
 1 out 
 (4)

	SPClamp{Multivariate} on Interface 1 (out) of Clamp{Multivariate} clamp_1
	SPClamp{MatrixVariate} on Interface 1 (out) of Clamp{MatrixVariate} clamp_2
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_1
	SPClamp{Multivariate} on Interface 1 (out) of Clamp{Multivariate} clamp_5
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} clamp_3
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} clamp_4
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_2
3.	SPMultiplicationOutVGP on Interface 1 (out) of Multiplication multiplication_1
4.	SPAdditionOutVGG on Interface 1 (out) of Addition addition_1



## Usage of composite nodes in an FFG

Now we 'draw a box' around the multiplication and addition nodes, and create a composite node. We can easily define a composite node using ForneyLab's `@composite` macro.

In [15]:
function func_Gain_Add(x,y)
    b = 0.5    
    return x + b*y
end

func_Gain_Add (generic function with 1 method)

In [16]:
# Define a composite node for z = x + b*y
@composite GainAddition (z, x, y) begin
    # Specify the 'internal factor graph' of the GainAddion composite node.
    # z, x, and y can be used as if they are existing Variables in this block.
    b = 0.5
    
    @RV z = func_Gain_Add(x,y)
end

Here, `GainAddition` is the name of the composite node that we're defining. The tuple `(z, x, y)` defines the variables that this node constrains. The order of these variables simultaneously fixes the argument order for the update rules. Now that our custom `GainAddition` composite node is defined, we can use it like any other factor node.

In [17]:
g2 = FactorGraph()

# Prior statistics
@RV mu_x_0
@RV Sigma_x_0 
@RV mu_u_1
@RV Sigma_u_1

@RV x_0 ~ GaussianMeanVariance(mu_x_0, Sigma_x_0)
@RV u_1 ~ GaussianMeanVariance(mu_u_1, Sigma_u_1)
@RV x_1 ~ GainAddition(x_0, u_1);

placeholder(mu_x_0, :mu_x_0)
placeholder(Sigma_x_0, :Sigma_x_0)
placeholder(mu_u_1, :mu_u_1)
placeholder(Sigma_u_1, :Sigma_u_1)
;

In [18]:
composite_schedule = sumProductSchedule(x_1)

draw(g2, schedule=composite_schedule)

println(composite_schedule)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.40.1 (20161225.0304)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 15824167123909048198 -->
 
 15824167123909048198 
 
 placeholder_mu_x_0 
 
<!-- 5719720873991217580 -->
 
 5719720873991217580 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 15824167123909048198--5719720873991217580 -->
 
 15824167123909048198--5719720873991217580 
 
 mu_x_0 
 2 m 
 1 out 
 
<!-- 8859035550829740761 -->
 
 8859035550829740761 
 
 𝒩 
 gaussianmeanvariance_2 
 
<!-- 10364190792435081580 -->
 
 10364190792435081580 
 
 placeholder_mu_u_1 
 
<!-- 10364190792435081580--8859035550829740761 -->
 
 10364190792435081580--8859035550829740761 
 
 mu_u_1 
 2 m 
 1 out 
 
<!-- 17204081788953040157 -->
 
 17204081788953040157 
 
 placeholder_Sigma_u_1 
 
<!-- 17204081788953040157--8859035550829740761 -->
 
 17204081788953040157--8859035550829740761 
 
 Sigma_u_1 
 3 v 
 1 out 
 
<!-- 11179237886781659611 -->
 
 11179237886781659611 
 
 GainAddition 
 gainaddition_1 
 
<!-- 11179237886781659611--8859035550829740761 -->
 
 11179237886781659611--8859035550829740761 
 
 u_1 
 1 out 
 (2) 
 3 y 
 
<!-- 11179237886781659611--5719720873991217580 -->
 
 11179237886781659611--5719720873991217580 
 
 x_0 
 1 out 
 (1) 
 2 x 
 
<!-- 880044161757782321 -->
 
 880044161757782321 
 
 placeholder_Sigma_x_0 
 
<!-- 880044161757782321--5719720873991217580 -->
 
 880044161757782321--5719720873991217580 
 
 Sigma_x_0 
 3 v 
 1 out 
 
<!-- 7334773912498534902 -->
 
 7334773912498534902 
 
<!-- 7334773912498534902--11179237886781659611 -->
 
 7334773912498534902--11179237886781659611 
 
 x_1 
 1 z 
 (3)

	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_Sigma_x_0
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_mu_x_0
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_1
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_Sigma_u_1
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_mu_u_1
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_2
3.	SPGainAdditionOutVGG on Interface 1 (z) of GainAddition gainaddition_1



Here, the resulting schedule contains one less message than before, because message (3) directly computes the belief over `x_1` from the prior beliefs. Currently, we have not defined a composite update rule yet and, as a default, message (3) is computed by internal message passing in the composite node. We can inspect this internal schedule.

In [19]:
println(composite_schedule[end].internal_schedule)

UndefRefError: UndefRefError: access to undefined reference

## Defining custom shortcut rules

If we actually want to use composite nodes to get a different (i.e. more efficient) algorithm, we'll have to specify message update rules that apply specifically to the composite nodes at hand. These rules are sometimes referred to as _shortcut rules_, since they provide a way to shortcut the calculation of internal messages. This rule definition for the composite nodes is analogous to the rule definitions for regular nodes. If we then build a new sum-product algorithm, the custom shortcut rule `SPGainAdditionOutVGG` will be automatically inserted.

In [20]:
@sumProductRule(:node_type     => GainAddition,                                 # our custom composite node
                :outbound_type => Message{GaussianMeanVariance},               # this rule produces a GaussianMeanPrecision msg
                :inbound_types => (Nothing, Message{Gaussian}, Message{Gaussian}), # msg towards first interface, incoming types
                :name          => SPGainAdditionOutVGG)                         # name of the update rule;

In [21]:
shortcut_schedule = sumProductSchedule(x_1)

println(shortcut_schedule)

	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_Sigma_x_0
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_mu_x_0
1.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_1
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_Sigma_u_1
	SPClamp{Univariate} on Interface 1 (out) of Clamp{Univariate} placeholder_mu_u_1
2.	SPGaussianMeanVarianceOutVPP on Interface 1 (out) of GaussianMeanVariance gaussianmeanvariance_2
3.	SPGainAdditionOutVGG on Interface 1 (z) of GainAddition gainaddition_1



In [22]:
algo = sumProductAlgorithm(x_1)
eval(Meta.parse(algo)) # Load algorithm

println(algo) # Inspect the algorithm

begin

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 3))

messages[1] = ruleSPGaussianMeanVarianceOutVPP(nothing, Message(Univariate, PointMass, m=data[:mu_x_0]), Message(Univariate, PointMass, m=data[:Sigma_x_0]))
messages[2] = ruleSPGaussianMeanVarianceOutVPP(nothing, Message(Univariate, PointMass, m=data[:mu_u_1]), Message(Univariate, PointMass, m=data[:Sigma_u_1]))
messages[3] = ruleSPGainAdditionOutVGG(nothing, messages[1], messages[2])

marginals[:x_1] = messages[3].dist

return marginals

end

end # block


In [23]:
function ruleSPGainAdditionOutVGG(msg_out::Nothing,msg_x::Message{F, V},msg_y::Message{F, V}) where {F<:Gaussian, V<:Univariate}

    x = convert(ProbabilityDistribution{V, GaussianMeanVariance}, msg_x.dist)
    y = convert(ProbabilityDistribution{V, GaussianMeanVariance}, msg_y.dist)
    b = 0.5

    Message(V, GaussianMeanVariance, m=x.params[:m]+b*y.params[:m], v=x.params[:v] + b^2 * y.params[:v])
end

ruleSPGainAdditionOutVGG (generic function with 1 method)

In [24]:
# Prior statistics
mu_x_0 = 1.0
Sigma_x_0 = 1.0
mu_u_1 = 1.0
Sigma_u_1 = 1.0

# Perform inference
#mu_x_0 = deepcopy(mu_x_0)
#Sigma_x_0 = deepcopy(Sigma_x_0)
data = Dict(:mu_x_0 => mu_x_0,
            :Sigma_x_0 => Sigma_x_0,
            :mu_u_1 => 1.0,
            :Sigma_u_1 => 1.0)
    
marginals = step!(data) # Execute the algorithm

Dict{Any,Any} with 1 entry:
  :x_1 => 𝒩(m=1.50, v=1.25)…